In [1]:
pip install pytrends

  Obtaining dependency information for pytrends from https://files.pythonhosted.org/packages/68/ba/7a24a3723c790000faf880505ff1cc46f4d29f46dd353037938a070c4d23/pytrends-4.9.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pytrends.request import TrendReq

# Initialiser Pytrends
pytrends = TrendReq(hl='fr', tz=360)  # 'fr' pour le français, fuseau horaire GMT+1


In [4]:
# Liste de mots-clés
keywords = ['Fake News', 'Théories du complot', 'Désinformation']


In [5]:
# Construire la requête
pytrends.build_payload(
    kw_list=keywords,  # Mots-clés
    cat=0,  # Catégorie 0 = Toutes les catégories
    timeframe='today 12-m',  # Période : Derniers 12 mois
    geo='FR',  # Région : France
    gprop=''  # Propriété : Recherche web (vide par défaut)
)


In [6]:
# Tendances temporelles
interest_over_time = pytrends.interest_over_time()
print(interest_over_time)


TooManyRequestsError: The request failed: Google returned a response with code 429

In [7]:
from pytrends.request import TrendReq
from pytrends.exceptions import TooManyRequestsError
import time

# Initialiser Pytrends avec des en-têtes personnalisés
pytrends = TrendReq(
    hl='fr',  # Langue française
    tz=360,  # Fuseau horaire (GMT+1)
    requests_args={
        'headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    }
)

# Liste de mots-clés
keywords_list = [['Fake News'], ['Théories du complot'], ['Désinformation']]

# Fonction pour gérer les requêtes et éviter les blocages
def fetch_trends(keywords, retries=3):
    """
    Récupère les données de tendances pour un ensemble de mots-clés
    en gérant les erreurs et les limitations.
    """
    for attempt in range(retries):
        try:
            # Construire la requête
            pytrends.build_payload(
                kw_list=keywords,
                cat=0,  # Toutes catégories
                timeframe='today 12-m',  # Derniers 12 mois
                geo='FR',  # France
                gprop=''  # Recherche web
            )
            # Extraire les tendances temporelles
            interest_over_time = pytrends.interest_over_time()
            if not interest_over_time.empty:
                print(f"Tendances récupérées pour {keywords}:")
                print(interest_over_time)
                return interest_over_time
            else:
                print(f"Aucune donnée pour {keywords}.")
                return None
        except TooManyRequestsError:
            print(f"Trop de requêtes. Tentative {attempt + 1}/{retries}. Pause de 2 minutes...")
            time.sleep(120)  # Pause de 2 minutes avant de réessayer
        except Exception as e:
            print(f"Erreur inattendue : {e}")
            break
    print(f"Échec après {retries} tentatives pour {keywords}.")
    return None

# Parcourir la liste des mots-clés et récupérer les données
all_trends = []
for keywords in keywords_list:
    trends = fetch_trends(keywords)
    if trends is not None:
        all_trends.append(trends)
    time.sleep(30)  # Pause de 30 secondes entre chaque mot-clé

# Exporter les données combinées en CSV
import pandas as pd
if all_trends:
    combined_data = pd.concat(all_trends, axis=1)
    combined_data.to_csv('tendances_google_trends.csv', index=True)
    print("Données exportées dans 'tendances_google_trends.csv'.")
else:
    print("Aucune donnée récupérée.")


Trop de requêtes. Tentative 1/3. Pause de 2 minutes...
Trop de requêtes. Tentative 2/3. Pause de 2 minutes...
Trop de requêtes. Tentative 3/3. Pause de 2 minutes...
Échec après 3 tentatives pour ['Fake News'].
Trop de requêtes. Tentative 1/3. Pause de 2 minutes...
Trop de requêtes. Tentative 2/3. Pause de 2 minutes...
Trop de requêtes. Tentative 3/3. Pause de 2 minutes...
Échec après 3 tentatives pour ['Théories du complot'].
Trop de requêtes. Tentative 1/3. Pause de 2 minutes...
Trop de requêtes. Tentative 2/3. Pause de 2 minutes...
Trop de requêtes. Tentative 3/3. Pause de 2 minutes...
Échec après 3 tentatives pour ['Désinformation'].
Aucune donnée récupérée.
